<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem1/blob/main/Evaluation_Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
data = pd.read_csv('/content/dataset.csv')
print(data.shape)

(160, 2)


In [ ]:
data.head()

,Predicted Class,Actual Class
0,0,0
1,0,0
2,1,0
3,0,0
4,0,0


In [ ]:
data['Predicted Class'].unique()

array([0, 1, 2, 3])

In [ ]:
data['Actual Class'].unique()

array([0, 1, 2, 3])

In [ ]:
data.shape[0]

160

##Confusion Matrix

In [ ]:
df_confusion = data.pivot_table(index=data['Actual Class'], columns=data['Predicted Class'], aggfunc=len).fillna(0).astype('int')
print(df_confusion)

Predicted Class   0   1   2   3
Actual Class                   
0                33   8   4   4
1                 1  32   3   0
2                 3   2  29   6
3                 2   3   3  27


## Overall Accuracy

In [ ]:
def overall_accuracy(df_confusion):
  sum_diag = 0
  for i in range(len(df_confusion)):
    sum_diag = df_confusion[i][i] + sum_diag
  overall_acc = sum_diag/ data.shape[0]
  #print(overall_acc)
  return overall_acc

## Class wise accuracy

In [ ]:
len(data['Predicted Class'].unique())

4

In [ ]:
def class_accuracy(class_name, df_confusion):
  sum = 0
  for i in range(len(df_confusion)):
    sum = df_confusion[i][class_name] + sum
  
  class_acc = df_confusion[class_name][class_name]/sum
  #print('class[{}], accuracy:{:.4f}' .format(class_name, class_acc))
  return class_acc


##Confusion Metrics

In [ ]:
def confusion_metrics(class_name, df_confusion):
  TN = 0
  FP = 0
  FN = 0
  TP = df_confusion[class_name][class_name]
  for i in range(len(df_confusion)):
    if i != class_name:
      TN = (np.sum(df_confusion[i]) + TN) - df_confusion[i][class_name]
      FP = df_confusion[class_name][i] + FP
      FN = df_confusion[i][class_name] + FN


  #print('True positive is {:.4f} for Class-->[{}]'.format(TP, class_name))
  #print('True Negative is {:.4f} for Class-->[{}]'.format(TN, class_name))
  #print('False Positive is {:.4f} for Class-->[{}]'.format(FP, class_name))
  #print('False Negative is {:.4f} for Class-->[{}]'.format(FN, class_name))

  return TP, TN, FP, FN

##Calculate Precision, Recall

In [ ]:
def calc_precision_recall(class_name):
  TP, TN, FP, FN = confusion_metrics(class_name, df_confusion)
  precision = TP/(TP+FP)
  recall = TP/(TP+FN)
  F_score = (2*precision*recall)/(precision + recall)

  #print('Precision is {:.4f} for class--> [{}]'.format(precision, class_name))
  #print('Recall is {:.4f} for class--> [{}]'.format(recall, class_name))
  #print('F-Score is {:.4f} for class--> [{}]'.format(F_score, class_name))

  return precision, recall, F_score


##Calculate Type-I and Type-II Error

In [ ]:
def calc_type1_type2(class_name):
  TP, TN, FP, FN = confusion_metrics(class_name, df_confusion)
  type1 = FP/(TN+FP)
  type2 = FN/(FN+TP)

  #print('Type-I error is {:.4f} for class--> [{}]'.format(type1, class_name))
  #print('Type-II error is {:.4f} for class--> [{}]'.format(type2, class_name))

  return type1, type2


##Calculate Macro and Weighted Average

In [ ]:
def macro_weighted_avg(df_confusion):
  history = []
  sum_list = []
  sum = 0
  for j in range(len(data['Predicted Class'].unique())):
    sum = 0
    for i in range(len(df_confusion)):
      sum = df_confusion[i][j] + sum
      history.append(calc_precision_recall(i))
    sum_list.append(sum)

  weighted_avg_prec = 0
  weighted_avg_recall = 0
  for i in range(len(sum_list)):
    weighted_avg_prec = sum_list[i]*history[i][0] + weighted_avg_prec
    weighted_avg_recall = sum_list[i]*history[i][1] + weighted_avg_recall
  weighted_avg_prec = weighted_avg_prec / (np.sum(sum_list))
  weighted_avg_recall = weighted_avg_recall / (np.sum(sum_list))

  macro_avg_prec = 0
  macro_avg_recall = 0
  for i in range(len(history)):
    macro_avg_prec = history[i][0] + macro_avg_prec
    macro_avg_recall = history[i][0] + macro_avg_recall
  macro_avg_prec = macro_avg_prec / len(history)
  macro_avg_recall = macro_avg_recall / len(history)

  return weighted_avg_prec, weighted_avg_recall, macro_avg_prec, macro_avg_recall

##Calculate all the Classification Metrices and print the results.

In [ ]:
def classification_metrics(class_name, df_confusion):
  overall_acc = overall_accuracy(df_confusion)
  class_acc = class_accuracy(class_name, df_confusion)
  TP, TN, FP, FN = confusion_metrics(class_name, df_confusion)
  precision, recall, F_score = calc_precision_recall(class_name)
  type1, type2 = calc_type1_type2(class_name)
  weighted_avg_prec, weighted_avg_recall, macro_avg_prec, macro_avg_recall = macro_weighted_avg(df_confusion)
  print('---------------------------------------')
  print('Overall Accuracy is : ',overall_acc)
  print('Weighted Average Precision : {:.4f}'.format(weighted_avg_prec))
  print('Weighted Average Recall : {:.4f}'.format(weighted_avg_recall))
  print('Macro Average Precision : {:.4f}'.format(macro_avg_prec))
  print('Macro Average Recall : {:.4f}'.format(macro_avg_recall))
  print('---------------------------------------')
  print('**Metrics for class {} are as below**'.format(class_name))
  print('---------------------------------------')
  print('Accuracy : {:.4f}' .format(class_acc))
  print('True positive : {:.4f} '.format(TP))
  print('True Negative : {:.4f} '.format(TN))
  print('False Positive : {:.4f} '.format(FP))
  print('False Negative : {:.4f} '.format(FN))
  print('Precision : {:.4f} '.format(precision))
  print('Recall : {:.4f}' .format(recall))
  print('F-Score : {:.4f}'.format(F_score))
  print('Type-I error : {:.4f}'.format(type1))
  print('Type-II error : {:.4f}'.format(type2))
  print('---------------------------------------')



In [ ]:
classification_metrics(0, df_confusion)

---------------------------------------
Overall Accuracy is :  0.75625
Weighted Average Precision : 0.7647
Weighted Average Recall : 0.7562
Macro Average Precision : 0.7576
Macro Average Recall : 0.7576
---------------------------------------
**Metrics for class 0 are as below**
---------------------------------------
Accuracy : 0.6735
True positive : 33.0000 
True Negative : 105.0000 
False Positive : 6.0000 
False Negative : 16.0000 
Precision : 0.8462 
Recall : 0.6735
F-Score : 0.7500
Type-I error : 0.0541
Type-II error : 0.3265
---------------------------------------


In [ ]:
classification_metrics(1, df_confusion)

---------------------------------------
Overall Accuracy is :  0.75625
Weighted Average Precision : 0.7647
Weighted Average Recall : 0.7562
Macro Average Precision : 0.7576
Macro Average Recall : 0.7576
---------------------------------------
**Metrics for class 1 are as below**
---------------------------------------
Accuracy : 0.8889
True positive : 32.0000 
True Negative : 111.0000 
False Positive : 13.0000 
False Negative : 4.0000 
Precision : 0.7111 
Recall : 0.8889
F-Score : 0.7901
Type-I error : 0.1048
Type-II error : 0.1111
---------------------------------------


In [ ]:
classification_metrics(2, df_confusion)

---------------------------------------
Overall Accuracy is :  0.75625
Weighted Average Precision : 0.7647
Weighted Average Recall : 0.7562
Macro Average Precision : 0.7576
Macro Average Recall : 0.7576
---------------------------------------
**Metrics for class 2 are as below**
---------------------------------------
Accuracy : 0.7250
True positive : 29.0000 
True Negative : 110.0000 
False Positive : 10.0000 
False Negative : 11.0000 
Precision : 0.7436 
Recall : 0.7250
F-Score : 0.7342
Type-I error : 0.0833
Type-II error : 0.2750
---------------------------------------


In [ ]:
classification_metrics(3, df_confusion)

---------------------------------------
Overall Accuracy is :  0.75625
Weighted Average Precision : 0.7647
Weighted Average Recall : 0.7562
Macro Average Precision : 0.7576
Macro Average Recall : 0.7576
---------------------------------------
**Metrics for class 3 are as below**
---------------------------------------
Accuracy : 0.7714
True positive : 27.0000 
True Negative : 115.0000 
False Positive : 10.0000 
False Negative : 8.0000 
Precision : 0.7297 
Recall : 0.7714
F-Score : 0.7500
Type-I error : 0.0800
Type-II error : 0.2286
---------------------------------------


## Confusion Matrix/ Metrics using sk-learn

In [ ]:
confusion_mtx = confusion_matrix(data['Actual Class'], data['Predicted Class']) 
print(confusion_mtx)

[[33  8  4  4]
 [ 1 32  3  0]
 [ 3  2 29  6]
 [ 2  3  3 27]]


In [ ]:
print(classification_report(data['Actual Class'], data['Predicted Class']))

              precision    recall  f1-score   support

           0       0.85      0.67      0.75        49
           1       0.71      0.89      0.79        36
           2       0.74      0.72      0.73        40
           3       0.73      0.77      0.75        35

    accuracy                           0.76       160
   macro avg       0.76      0.76      0.76       160
weighted avg       0.76      0.76      0.76       160



##Hamming Loss

In [ ]:
from sklearn.metrics import hamming_loss
hamming_loss(data['Actual Class'], data['Predicted Class'])

0.24375

##Matthews Correlation Coefficient

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(data['Actual Class'], data['Predicted Class'])

0.6785227135925035

##Zero-one Loss

In [ ]:
from sklearn.metrics import zero_one_loss
zero_one_loss(data['Actual Class'], data['Predicted Class'], normalize = False)

39